## catg testing

In [23]:
import ipyrad as ip
import ipyparallel
import subprocess
import os

In [24]:
## load in a data set that is big, i.e., it has a large 
## {projdir}/{ccnsensdir}/{name}_cathaps.tmp and thus will take a long
## time to cluster, so it prints a progress bar slowly over time

data = ip.load_json("/home/deren/Downloads/pedicularis/cyatho-min8.json")

  loading Assembly: cyatho-min8
  from saved path: ~/Downloads/pedicularis/cyatho-min8.json


In [25]:
def cluster(data):
    """ returns a cmd list """
    
-

    #proc = subprocess.Popen(cmd, stderr=subprocess.STDOUT, stdout=subprocess.PIPE)
    #proc.wait()

In [26]:
## store the command list
cmd = cluster(data)

### Now run it
Our goal is to run it in a way that we can read the output to stderr 
that is in the form of (Clustering XX%) and parse it so that we can 
create a progress bar similar to the other steps. 

#### Attempt 1:
use files like in: http://stackoverflow.com/questions/18421757/live-output-from-subprocess-command

In [18]:
with open('test.log', 'w') as f:
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    for c in iter(lambda: process.stdout.read(1), ''):
        if c == '\r':
            print(c)
            f.write("".join(acc))
            acc = []
        else:
            acc.append(c)
        f.write("".join(acc))

KeyboardInterrupt: 

In [ ]:
m, s = pty.openpty()
fm = os.fdopen(m, "rw")
p = subprocess.Popen(["python2", "test.py"], stdin=s, stdout=s, stderr=s)
p.communicate()
os.close(s)
print fm.read()

In [ ]:
from select import select
master, slave = pty.openpty()

process = subprocess.Popen(cmd, stderr=writer)
pin = os.fdopen(master, 'w')
tty.setcbreak(sys.stdin)

while True:
    rs, wc, es = select([])

    while process.poll() is None:
        sys.stdout.write(reader.read())
        sys.stdout.flush()
        time.sleep(0.5)
    # Read the remaining
    sys.stdout.write(reader.read())

In [ ]:
import io
import time
import subprocess
import sys

filename = 'test.log'

with io.open(filename, 'wb') as writer, io.open(filename, 'rb', 1) as reader:
    process = subprocess.Popen(cmd, stderr=writer)
    while process.poll() is None:
        print(is)
        for c in iter(lambda: reader.read(), ""):
            if c == "\r":
                print("".join(acc))
                acc = []
            else:
                acc.append(c)
        time.sleep(0.5)
    # Read the remaining
    sys.stdout.write(reader.read())

In [17]:
import io
import time
import subprocess
import sys

filename = 'test.log'
with io.open(filename, 'wb') as writer, io.open(filename, 'rb', 1) as reader:
    process = subprocess.Popen(cmd, stderr=writer)#, stdout=writer)
    while process.poll() is None:
        linedat = reader.read().strip()
        #writer.write(linedat)
        if "Clustering" in linedat:
            prog = linedat.split()
            print prog
            #ip.assemble.util.progressbar(100, prog, 
            #            " clustering across 1/4  | {}".format('running...'))
        time.sleep(0.5)
    # Read the remaining
    linedat = reader.read().strip()
    if "Clustering" in linedat:
        prog = 100#int(linedat.split()[1][:-1])
        print prog

KeyboardInterrupt: 

In [6]:
ipyclient = ipyparallel.Client()

In [7]:
res = ipyclient[0].apply(cluster, data)

In [10]:
res.display_outputs

<bound method AsyncResult.display_outputs of <AsyncResult: finished>>

In [12]:
print res.ready()
res.metadata

True


{'after': [],
 'completed': datetime.datetime(2016, 4, 26, 17, 57, 53, 945046),
 'data': {},
 'engine_id': 0,
 'engine_uuid': u'66a27d14-7a7a-4ed1-bd2a-f6691807408f',
 'error': ipyparallel.error.RemoteError(u'UnsupportedOperation',
                               u'IOStream has no fileno.'),
 'execute_input': None,
 'execute_result': None,
 'follow': [],
 'msg_id': u'55f60151-3278-4ae7-af8c-b0419716748b',
 'outputs': [],
 'received': datetime.datetime(2016, 4, 26, 17, 57, 53, 950281),
 'started': datetime.datetime(2016, 4, 26, 17, 57, 52, 713298),
 'status': u'error',
 'stderr': '',
 'stdout': '',
 'submitted': datetime.datetime(2016, 4, 26, 17, 57, 52, 708782)}

In [8]:
io5 = h5py.File(data.database, 'r')
io5.keys()

[u'catgs', u'edges', u'filters', u'seqs', u'snps']

In [9]:
io5['catgs'].attrs['samples']

array(['1A_0', '1B_0', '1C_0', '1D_0', '2E_0', '2F_0', '2G_0', '2H_0',
       '3I_0', '3J_0', '3K_0', '3L_0'], 
      dtype='|S4')

In [13]:
io5['seqs'][1, :, 65:76]

array([['A', 'C', 'C', 'G', 'A', 'A', 'G', 'C', 'G', 'T', 'T'],
       ['A', 'C', 'C', 'G', 'A', 'A', 'G', 'C', 'G', 'T', 'T'],
       ['A', 'C', 'C', 'G', 'A', 'A', 'G', 'C', 'G', 'T', 'T'],
       ['A', 'C', 'C', 'G', 'A', 'A', 'G', 'C', 'G', 'T', 'T'],
       ['A', 'C', 'C', 'G', 'A', 'A', 'G', 'C', 'G', 'T', 'T'],
       ['A', 'C', 'C', 'G', 'A', 'A', 'G', 'C', 'G', 'T', 'T'],
       ['A', 'C', 'C', 'G', 'A', 'A', 'G', '-', 'G', 'T', 'T'],
       ['A', 'C', 'C', 'G', 'A', 'A', 'G', 'C', 'G', 'T', 'T'],
       ['A', 'C', 'C', 'G', 'A', 'A', 'G', 'C', 'G', 'T', 'T'],
       ['A', 'C', 'C', 'G', 'A', 'A', 'G', 'C', 'G', 'T', 'T'],
       ['A', 'C', 'C', 'G', 'A', 'A', 'G', 'C', 'G', 'T', 'T'],
       ['A', 'C', 'C', 'G', 'A', 'A', 'G', 'C', 'G', 'T', 'T']], 
      dtype='|S1')

In [14]:
io5['catgs'][1, :, 65:76]

array([[[ 0, 26,  0,  0],
        [26,  0,  0,  0],
        [26,  0,  0,  0],
        [ 0,  0,  0, 26],
        [ 0, 25,  0,  1],
        [ 0, 26,  0,  0],
        [ 0,  0,  0, 26],
        [26,  0,  0,  0],
        [ 0,  0,  0, 26],
        [ 0,  0, 26,  0],
        [ 0,  0, 26,  0]],

       [[ 0, 21,  0,  0],
        [21,  0,  0,  0],
        [20,  0,  1,  0],
        [ 0,  0,  0, 21],
        [ 0, 21,  0,  0],
        [ 0, 21,  0,  0],
        [ 0,  0,  0, 21],
        [21,  0,  0,  0],
        [ 0,  0,  0, 21],
        [ 0,  0, 21,  0],
        [ 0,  0, 21,  0]],

       [[ 0, 20,  0,  0],
        [20,  0,  0,  0],
        [20,  0,  0,  0],
        [ 0,  0,  0, 20],
        [ 0, 20,  0,  0],
        [ 0, 20,  0,  0],
        [ 0,  0,  0, 20],
        [20,  0,  0,  0],
        [ 0,  0,  0, 20],
        [ 0,  0, 20,  0],
        [ 0,  0, 20,  0]],

       [[ 0, 16,  0,  0],
        [16,  0,  0,  0],
        [16,  0,  0,  0],
        [ 0,  0,  0, 16],
        [ 0, 16,  0,  0],
      

In [23]:
import ipyparallel as ipp
c = ipp.Client()
lbview1 = c.load_balanced_view()
lbview2 = c.load_balanced_view()

In [24]:
def returnsleep(x):
    import time
    time.sleep(x)
    return "that felt good"

In [28]:
import time
import itertools

job1 = {}
for i in range(4):
    job1[i] = lbview1.apply(time.sleep, 2)

job2 = {}
for i in range(4):
    with lbview1.temp_flags(after=job1[i]):
        job2[i] = lbview1.apply(time.sleep, 2)

job3 = {}
for i in range(4):
    with lbview1.temp_flags(after=job2[i]):
        job3[i] = lbview1.apply(time.sleep, 2)

job4 = {}
for i in range(4):
    with lbview1.temp_flags(after=job3[i]):
        job4[i] = lbview1.apply(returnsleep, 2)

        
waitfor = list(itertools.chain(*[i.msg_ids for i in job4.values()]))
print waitfor

with lbview1.temp_flags(after=waitfor):
    res = lbview1.apply(time.sleep, 2)

['a237079b-0737-477d-9a82-c84a66e827b6', '3a2663a0-defd-4dbc-a0ae-8fcfddc6b04e', '098eeee7-e64c-4efe-bcc2-beffefde3a3e', '9e3cad74-4756-4bc5-8da6-2b13598f64f9']


In [29]:
while 1:
    if not res.ready():
        print sum([job2[i].ready() for i in job2])
        print sum([job4[i].ready() for i in job4])
        time.sleep(1)
    else:
        break

0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
4
0
4
0
4
0
4
4
4
4


### Running sequential jobs w/ dependencies

In [14]:
c.wait_interactive()
c.

  17/17 tasks finished after   10 s
done


### Retrieving returned results

In [28]:
res = job4[0]
print res.result()
print res.successful()

that felt good
True


### Retrieving returned results AND raise error if failed

In [33]:
while 1:
    try:
        if all([job4[i].successful() for i in job4]):
            break
    except AssertionError:
        pass
    print 'hi', '[#####]', "time"
    time.sleep(1)

hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time


### Running a non-blocking wait call

In [22]:
while not all([job4[i].ready() for i in job4]):
    print 'hi', '[#####]', "time"
    time.sleep(1)

hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time


In [36]:
res.elapsed

20.041482

In [34]:
import numpy as np
from collections import Counter

a = np.array([0, 1, 2, 5, 3, 3, 3, 4])
dict(Counter(a))

{0: 1, 1: 1, 2: 1, 3: 3, 4: 1, 5: 1}

In [38]:
a
b = [0, 1, 2]

[i for i in a if i in b]

[0, 1, 2]

In [ ]:
### profiling
python -m cProfile -s cumulative ../ipyrad/assemble/cluster_within.py
